In [ ]:
!pip install nltk textblob spacy transformers

import spacy
from textblob import TextBlob
from transformers import pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import pandas as pd
from google.colab import drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:

# Print version information
print("Spacy version:", spacy.__version__)
print("TextBlob version:", textblob.__version__)
print("Transformers version:", transformers_version)
print("Scikit-Learn version:", sklearn.__version__)
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)


Spacy version: 3.7.5
TextBlob version: 0.17.1
Transformers version: 4.46.2
Scikit-Learn version: 1.5.2
NumPy version: 1.26.4
Pandas version: 2.2.2


In [ ]:
file_url = f'/Updated Dataset (1).xlsx'

data = pd.read_excel(file_url)


NameError: name 'pd' is not defined

In [ ]:
nlp = spacy.load('en_core_web_sm')
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

features = data.drop(columns=['Movement', 'Duration of Movement'])
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def calculate_individual_theme_score(text, label):
    result = classifier(text, candidate_labels=[label])
    score = result['scores'][0] * 100
    return score

def extract_features_from_text(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity

    positive_sentiment = max(0, polarity) * 50 + 50 * (1 - abs(polarity))
    negative_sentiment = max(0, -polarity) * 50 + 50 * (1 - abs(polarity))
    neutral_sentiment = 100 - (positive_sentiment + negative_sentiment)

    positive_sentiment = (positive_sentiment / (positive_sentiment + negative_sentiment + neutral_sentiment)) * 100
    negative_sentiment = (negative_sentiment / (positive_sentiment + negative_sentiment + neutral_sentiment)) * 100
    neutral_sentiment = 100 - (positive_sentiment + negative_sentiment)

    political_nature = calculate_individual_theme_score(text, 'political')
    counter_movement = calculate_individual_theme_score(text, 'counter movement')
    cultural_relevance = calculate_individual_theme_score(text, 'cultural')
    legality_nature = calculate_individual_theme_score(text, 'legal')

    features = np.array([political_nature, counter_movement, cultural_relevance, legality_nature,
                         positive_sentiment, negative_sentiment, neutral_sentiment]).reshape(1, -1)
    return features


In [ ]:
def find_similar_movements(features, top_n=3):
    scaled_new_data = scaler.transform(features)
    distances = euclidean_distances(scaled_features, scaled_new_data)
    similar_indices = np.argsort(distances.flatten())[:top_n]
    similar_movements = data.iloc[similar_indices]['Movement']
    return similar_movements

In [ ]:
text_input = "The recent protest for climate change has gained massive global attention, urging governments to take legal actions to protect the environment."
features_from_text = extract_features_from_text(text_input)
similar_movements = find_similar_movements(features_from_text)

print("Extracted features from text:", features_from_text)
print("Most similar movements to the provided text:")
print(similar_movements.to_list())

Extracted features from text: [[95.23722529  0.11654415  0.19965116  3.40713374 50.         47.5
   2.5       ]]
Most similar movements to the provided text:
['NoDapl ', 'standwithhongkong', 'FightFor15']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
import pickle

# Assuming best_rf is your model that you want to serialize
best_rf = ...  # Define or load your model here

with open('TextToMovementDetails.pkl', 'wb') as file:
    pickle.dump(best_rf, file)